#### 5. Система бронирования авиакомпаний/отелей. Создайте систему бронирования, которая бронирует места в самолетах или номера в отелях. Он взимает различные тарифы за определенные участки самолета или отеля. Например, первый класс будет стоить больше, чем эконом. В гостиничных номерах есть пентхаусы, которые стоят дороже. Следите за тем, когда комнаты будут доступны и могут быть запланированы.

In [149]:
from enum import Enum
from typing import TypedDict


class RoomClass(Enum):  # класс гостиницы
    Standart = "standart"
    Penthouse = "penthouse"

    def __repr__(self):
        return self.value


class RoomReservation(TypedDict):
    hotel_title: str
    room_class: RoomClass
    date: str


class Hotel:
    def __init__(self, title: str, rooms: dict[RoomClass, int]):
        self.title = title
        self.rooms = {RoomClass.Standart: 0, RoomClass.Penthouse: 0, **rooms}

    def __repr__(self):
        return f'Hotel {self.title}, Vacant spaces: {self.rooms}'


class Supervisor:
    def __init__(self, hotels: list[Hotel] = []):
        self._hotels = hotels

        self._room_reservations: list[RoomReservation] = []

    def add_hotel(self, hotel: Hotel):
        if hotel in self._hotels:
            return print(f'Oops \'{hotel.title}\' already on the list.')
        self._hotels += [hotel]

    def remove_hotel(self, hotel_title: str):
        hotel_to_remove = self._find_hotel(hotel_title)

        if hotel_to_remove is None:
            return print(f'There\'s no hotel {hotel_title} on the list.')

        self._hotels.remove(hotel_to_remove)
        print(f'Removed hotel {hotel_title}')

    def get_hotels(self):
        return self._hotels

    def book_a_room(self, hotel_title: str, room_class: RoomClass, date: str):
        hotel_to_book = self._find_hotel(hotel_title)
        if hotel_to_book is None:
            return print(f'There\'s no hotel \'{hotel_title}\' on the list.')

        has_vacant_rooms, vacant_spaces_count = self._has_vacant_rooms(
            hotel_to_book, room_class, date)
        if has_vacant_rooms:
            self._room_reservations += [
                self._create_room_reservation(hotel_title, room_class, date)]
            return print(
                f'Booked a room \'{room_class.value}\' in \'{hotel_title}\' on {date}. {vacant_spaces_count - 1} left on this date.')

        print(
            f'No vacant rooms \'{room_class.value}\' in {hotel_title} on {date}.')

    def _create_room_reservation(self, hotel_title: str, room_class: RoomClass, date: str) -> RoomReservation:
        return {"hotel_title": hotel_title, "room_class": room_class, "date": date}

    def _find_hotel(self, hotel_title: str) -> Hotel | None:
        return next(
            (hotel for hotel in self._hotels if hotel.title == hotel_title), None)

    def _has_vacant_rooms(self, hotel: Hotel, room_class: RoomClass, date: str) -> tuple[bool, int]:
        this_day_reservations = list(filter(
            lambda r: r["hotel_title"] == hotel.title and r["date"] == date and r["room_class"] == room_class, self._room_reservations))

        vacant_spaces_count = hotel.rooms[room_class] - \
            len(this_day_reservations)
        has_vacant_rooms = vacant_spaces_count > 0

        return (has_vacant_rooms, vacant_spaces_count)


mariott = Hotel("Mariott", {RoomClass.Standart: 3, RoomClass.Penthouse: 2})

sv = Supervisor(
    [mariott, Hotel("Radisson", {RoomClass.Standart: 3, RoomClass.Penthouse: 5})])

sv.book_a_room(mariott.title, RoomClass.Standart, "24.10.2022")
sv.book_a_room(mariott.title, RoomClass.Standart, "24.10.2022")
sv.book_a_room(mariott.title, RoomClass.Standart, "24.10.2022")
sv.book_a_room(mariott.title, RoomClass.Standart, "24.10.2022")
sv.book_a_room(mariott.title, RoomClass.Penthouse, "24.10.2022")

sv.book_a_room("Radisson", RoomClass.Standart, "24.10.2022")
sv.book_a_room("Radisson", RoomClass.Penthouse, "24.10.2022")
sv.book_a_room("Radisson", RoomClass.Penthouse, "25.10.2022")


Booked a room 'standart' in 'Mariott' on 24.10.2022. 2 left on this date.
Booked a room 'standart' in 'Mariott' on 24.10.2022. 1 left on this date.
Booked a room 'standart' in 'Mariott' on 24.10.2022. 0 left on this date.
No vacant rooms 'standart' in Mariott on 24.10.2022.
Booked a room 'penthouse' in 'Mariott' on 24.10.2022. 1 left on this date.
Booked a room 'standart' in 'Radisson' on 24.10.2022. 2 left on this date.
Booked a room 'penthouse' in 'Radisson' on 24.10.2022. 4 left on this date.
Booked a room 'penthouse' in 'Radisson' on 25.10.2022. 4 left on this date.
